In [ ]:
using DiffusionMLE, Plots

# Introduction

This is a minimal example on how to apply our maximum likelihood estimator to a set of homogeneous single-particle tracking data.  Here, "homogeneous" refers to the fact that the underlying diffusive dynamics is the same for all trajectories.  

For more details on the theoretical framework, please refer to the associated preprint:
> J. T. Bullerjahn and G. Hummer, "Maximum likelihood estimates of diffusion coefficients from single-molecule tracking experiments", https://arxiv.org/abs/2011.09955

# Generate trajectories

Each trajectory can be seen as a $d$-dimensional array (<code>Array{Float64,2}</code>), so the data set should be of the type <code>Array{Array{Float64,2},1}</code>.  

Here, we generate mock data, made up of $M$ $d$-dimensional trajectories of different lengths $N = \{N_{1}, N_{2}, \dots, N_{M}\}$.  The $N_{i}$ are distributed uniformly on the interval $[3,100]$.  

In [ ]:
include("smeared_trajectory_integrator.jl")

const M = 1000 # Number of trajectories
const d = 2 # Dimension of trajectories

const N_sub = 100 # Number of substeps over which the trajectory is smeared out

N = [ rand(3:100) for i = 1 : M ] # Array of trajectory lengths

const a2 = 1.2
const σ2 = 2.3

B = [1/6 for m = 1 : M] # Array of blurring coefficients, where we have assumed a uniform illumination profile
data = make_2D_data(N,N_sub,a2,σ2); # Mock data set

# Analyzing the data

### Parameter estimation

In [ ]:
parameters = MLE_estimator(B,data)
uncertainties = MLE_errors(B,data,parameters)

println("Estimates:")
println(string("a2 = ", parameters[1], " ± ", uncertainties[1]))
println(string("σ2 = ",parameters[2], " ± ", uncertainties[2]))

println()

println("Ground truth:")
println(string("a2 = ", a2))
println(string("σ2 = ", σ2))

### Diffusion coefficients

Diffusion coefficients can be extracted from the $\sigma^2$-values, irrespective of the dimension $d$, as follows:
\begin{equation*}
D = \frac{\sigma^2}{2 \Delta t} \, .  
\end{equation*}
Here, $\sigma$ has the same dimension as the data, i.e., if the trajectories are recorded on the nanometer scale then $[\sigma] = \textrm{nm}$, and $\Delta t$ denotes the time step between two observations.  

# Testing model assumptions

We have assumed that the data are homogeneous.  To test this and other assumptions of our diffusion model (Gaussianity, structure of covariance matrix, etc.), we perform a quality factor analysis:

In [ ]:
Q = Q_factor_analysis(B,data,parameters)

println("Kuiper statistic:")
println(string("κ = ", Kuiper_statistic!(Q)))

histogram(Q, normed=true, legend=false, 
    xlabel="quality factor Q", 
    ylabel="probability distribution function p(Q)")

The uniform distribution of the quality factors, as well as the fact that the Kuiper statistic evaluates close to unity, indicate that the assumptions underlying the model are satisfied.  This is, of course, not very surprising, because the mock data was generated with said assumptions in mind.  